In [204]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Reading our Immoweb dataset in 'df'.
df = pd.read_csv('../data/merged_data.csv')

# Instantiating LinearRegression as 'reg'.
reg = LinearRegression()

# Filling NaN values with 0.
df['landplot'].fillna(0, inplace=True)
df['facades'].fillna(0, inplace=True)
df['Living area'].fillna(0, inplace=True)

# Creating dummy columns from categorical data.
ello = pd.get_dummies(data=df, columns=['condition', 'subtype', 'province', 'Zip code'])
df.drop(['type', 'condition', 'province', 'subtype', 'Zip code', 'city', 'facades', 'Terrace', 'Kitchen'], axis=1, inplace=True)
df = df.join(ello)
'''df = df.join(pd.get_dummies(df['subtype']).astype(int))
df = df.join(pd.get_dummies(df['province']).astype(int))
df = df.join(pd.get_dummies(df['Zip code']).astype(int))'''
#df = df.join(pd.get_dummies(df['city'], ).astype(int))


display(df.head(10))
#sns.heatmap(df.corr(), annot=True, cmap='Purples')

# Defining 'X' and 'y' variables from our dataframe using purely features that contain numerical data.
X = df.drop(['price'], axis=1).to_numpy()
y = df['price'].to_numpy()

# Reshaping 'y' to be 2D array.
y = y.reshape(-1, 1)

# Setting up 'train_test_split' to get standardized training/testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Training our model.
reg.fit(X_train, y_train)

# Displaying score of Training variables.
print("Training score:", reg.score(X_train, y_train)) 

# Predicting the 'y' target value (Price).
y_prediction = reg.predict(X_test)
# Displaying the score of Testing variables
print("Testing score:", reg.score(X_test, y_test))
print("Using 7 features; 'Bedroom count', 'Living surface area', 'Landplot area', 'Housing type', 'Condition', 'Province, and 'Terrace' and 1 (price)target")


ValueError: columns overlap but no suffix specified: Index(['price', 'bedrooms', 'Living area', 'landplot'], dtype='object')